## TODO

* HAPPENED_AT for police calls
  * PD

In [2]:
# import libraries
from pathlib import Path
import string
import pandas as pd
import sys
sys.path.insert(0, '../data_cleaning_and_integration')

from cleaner_lib import remove_puncuations

In [3]:
# set paths
data_p             = Path("../data")
pd_processed_p = data_p / "processed_pd_data"

# create out path
out_p = data_p / "neo4j_files"
out_p.mkdir(exist_ok=True)

node_p = out_p / "nodes"
node_p.mkdir(exist_ok=True)

relations_p = out_p / "relationships"
relations_p.mkdir(exist_ok=True)

### Time of day nodes

Police Crime nodes

In [89]:
pd_df = pd.read_csv(pd_processed_p / 'merged_pd_data.csv')

In [90]:
time_node_df = pd_df.copy(deep = True)
time_node_df[":LABEL"] = "TIME_OF_DAY"

#droping duplicates
time_node_df = time_node_df[['date_time_bin',':LABEL']]

time_node_df.drop_duplicates(inplace=True)
time_node_df.reset_index(inplace = True, drop = True)
time_node_df['time_of_day_id:ID'] = time_node_df.index
time_node_df['time_of_day_id:ID'] = 'tm' + time_node_df['time_of_day_id:ID'].astype('str')
time_node_df = time_node_df[['time_of_day_id:ID', 'date_time_bin',':LABEL']]

time_node_df.to_csv(node_p / 'time_of_day_nodes.csv',index=False)
time_node_df

,time_of_day_id:ID,date_time_bin,:LABEL
0,tm0,Late Night,TIME_OF_DAY
1,tm1,Early Morning,TIME_OF_DAY
2,tm2,Morning,TIME_OF_DAY
3,tm3,Noon,TIME_OF_DAY
4,tm4,Evening,TIME_OF_DAY
5,tm5,Night,TIME_OF_DAY


In [92]:
pd_time_rel =  pd_df.merge(time_node_df, left_on = 'date_time_bin', right_on = 'date_time_bin')
pd_time_rel = pd_time_rel[['incident_num', 'time_of_day_id:ID']]
pd_time_rel.rename(columns = {'incident_num': 'incident_num:START_ID', 'time_of_day_id:ID': ':END_ID'}, inplace = True)
pd_time_rel[':TYPE'] = 'HAPPENED_AT'
pd_time_rel

,incident_num:START_ID,:END_ID,:TYPE
0,E22010000006,tm0,HAPPENED_AT
1,E22010000007,tm0,HAPPENED_AT
2,E22010000009,tm0,HAPPENED_AT
3,E22010000012,tm0,HAPPENED_AT
4,E22010000014,tm0,HAPPENED_AT
...,...,...,...
103245,E22050033473,tm5,HAPPENED_AT
103246,E22050033474,tm5,HAPPENED_AT
103247,E22050033475,tm5,HAPPENED_AT
103248,E22050033476,tm5,HAPPENED_AT


Merge with the node dataset to make sure there are no relationships on non existant nodes

In [83]:
pd_nodes = pd.read_csv(node_p / 'pd_crime_nodes.csv')

In [93]:
rel = pd_time_rel.merge(pd_nodes, left_on = 'incident_num:START_ID', right_on = 'incident_num:ID', how = 'inner')[['incident_num:START_ID', ':END_ID', ':TYPE']]
rel

,incident_num:START_ID,:END_ID,:TYPE
0,E22010000006,tm0,HAPPENED_AT
1,E22010000007,tm0,HAPPENED_AT
2,E22010000009,tm0,HAPPENED_AT
3,E22010000012,tm0,HAPPENED_AT
4,E22010000014,tm0,HAPPENED_AT
...,...,...,...
103245,E22050033473,tm5,HAPPENED_AT
103246,E22050033474,tm5,HAPPENED_AT
103247,E22050033475,tm5,HAPPENED_AT
103248,E22050033476,tm5,HAPPENED_AT


In [94]:
rel.to_csv(relations_p / 'police_HA_rels.csv', index = False)